In [1]:
from google.colab import drive
from google.colab import files
import os
import pandas as pd
import io
drive.mount('/content/drive')
#!pip install tensorflow
#!pip install keras
#!pip install numpy

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!unzip /content/drive/MyDrive/donnees.zip

Archive:  /content/drive/MyDrive/donnees.zip
replace donnees/avis.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#print ('/content/drive/MyDrive/donnees/avis.csv')
with open('donnees/categories.csv', 'r') as categories_file:
    #print(f.read()[:10000] + '...')

    #df1 = pd.read_csv(io.StringIO(f.read()))
    categories = pd.read_csv(categories_file, skipinitialspace=True)
    #print(categories)
#data_path = '/content/drive/MyDrive/donnees/'

with open('donnees/avis.csv', 'r') as avis_file:
    avis = pd.read_csv(avis_file, skipinitialspace=True)
    #print(avis)

with open('donnees/checkin.csv', 'r') as checkin_file:
    checkin = pd.read_csv(checkin_file, skipinitialspace=True)
    #print(checkin)

with open('donnees/conseils.csv', 'r') as conseils_file:
    conseils = pd.read_csv(conseils_file, skipinitialspace=True)
    #print(conseils)

with open('donnees/horaires.csv', 'r') as horaires_file:
    horaires = pd.read_csv(horaires_file, skipinitialspace=True)
    #print(horaires)

with open('donnees/restaurants.csv', 'r') as restaurants_file:
    restaurants = pd.read_csv(restaurants_file, skipinitialspace=True)
    #print(restaurants)


with open('donnees/services.csv', 'r') as services_file:
    services = pd.read_csv(services_file, skipinitialspace=True)
    #print(services)

with open('donnees/utilisateurs.csv', 'r') as utilisateurs_file:
    utilisateurs = pd.read_csv(utilisateurs_file, skipinitialspace=True)
    #print(utilisateurs)

In [ ]:
print("Taille des données:")
print("------------------")
print("utilisateurs:\t", len(utilisateurs))
print("avis:\t\t\t", len(avis))
print("conseils:\t\t", len(conseils))
print("checkin:\t\t", len(checkin))
print("restaurants:\t", len(restaurants))
print("horaires:\t\t", len(horaires))
print("services:\t\t", len(services))
print("categories:\t\t", len(categories))
print("------------------")

Taille des données:
------------------
utilisateurs:	 748247
avis:			 2073685
conseils:		 416803
checkin:		 5492282
restaurants:	 34268
horaires:		 34268
services:		 34268
categories:		 89945
------------------


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ==========================================
# ==========CALCUL DES FEATURES=============
# ==========================================

# Initialisation du Dataframe "features" qui va contenir l'ensemble de données d'entrainement
# -----------------------------------------------------
# restaurant_id:  C'est l'identifiant (ID) du restaurant
# ferme: C'est la classe (Fermeture définitive : 1 , ouvert: 0)
# Les 3 premiers features sont déjà chargés: moyenne_etoiles, ville et zone
features = restaurants[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme']].copy()

# Extraire l'année à partir des dates
# La colonne "annee" sera utilisée dans le calcul de certain features
# -----------------------------------------------------
checkin["date"] = pd.to_datetime(checkin["date"], format='%Y-%m-%d')
checkin['annee'] = checkin.date.dt.year

avis["date"] = pd.to_datetime(avis["date"], format='%Y-%m-%d')
avis['annee'] = avis.date.dt.year

conseils["date"] = pd.to_datetime(conseils["date"], format='%Y-%m-%d')
conseils['annee'] = conseils.date.dt.year

In [ ]:
# -----------------------------------------------------------
# 4) nb_restaurants_zone
# Le nombre de restaurants dans la zone associée au restaurant en question.
# -----------------------------------------------------------

# Groupement par 'zone' et comptage des restaurants
nb_restaurants_zone = restaurants.groupby('zone').size().reset_index(name='nb_restaurants_zone')
# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(nb_restaurants_zone, on='zone', how='left')
features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone']].copy()
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0
...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0


In [ ]:


# -----------------------------------------------------------
# 5) zone_categories_intersection
# Le nombre de restaurants dans la même zone qui partagent au moins une catégorie avec le restaurant en question.
# -----------------------------------------------------------

# Fusion de 'restaurants' et 'categories' puis groupement par 'zone' et 'categorie'
restaurants_categories = restaurants[['restaurant_id', 'zone']].merge(categories, on='restaurant_id', how='left')
zone_categories_count = restaurants_categories.groupby(['zone', 'categorie']).size().reset_index(name='count')
# Calcul du nombre total de catégories partagées par restaurant
zone_categories_total = restaurants_categories.merge(zone_categories_count, on=['zone', 'categorie'], how='left')
zone_categories_total = zone_categories_total.groupby('restaurant_id')['count'].sum().reset_index(name='zone_categories_intersection')
zone_categories_total = zone_categories_total[zone_categories_total['zone_categories_intersection'] > 1]
zone_categories_total
# Fusion avec le DataFrame 'features' pour ajouter cette information
#features = features.merge(zone_categories_total, on='restaurant_id', how='left')

,restaurant_id,zone_categories_intersection
0,--1UhMGODdWsrMastO9DZw,6.0
1,--6MefnULPED_I942VcFNA,77.0
2,--DaPTJW3-tB1vP-PfdTEg,3.0
3,--FBCX-N37CMYDfs790Bnw,76.0
4,--KCl2FvVQpvjzmZSPyviA,26.0
...,...,...
34262,zze6IysT7bJFS8gvi6fZ2A,34.0
34263,zzjKekzQ6i4iR-qpo405Pw,3.0
34265,zzsU528uoRB6qZUGhKDa6w,17.0
34266,zzvlwkcNR1CCqOPXwuvz2A,104.0


In [ ]:
# -----------------------------------------------------------
# 6) ville_categories_intersection
# Le nombre de restaurants dans la même ville qui partagent au moins une catégorie avec le restaurant en question.
# -----------------------------------------------------------

# Fusion entre les restaurants et leurs catégories
restaurants_ville_categories = restaurants[['restaurant_id', 'ville']].merge(categories, on='restaurant_id', how='left')
# Groupement par ville et catégorie pour compter le nombre d'occurrences
ville_categories_count = restaurants_ville_categories.groupby(['ville', 'categorie']).size().reset_index(name='count')
#ville_categories_count
# Calcul du nombre total de catégories partagées par restaurant dans la même ville
ville_categories_total = restaurants_ville_categories.merge(ville_categories_count, on=['ville', 'categorie'], how='left')
ville_categories_total = ville_categories_total.groupby('restaurant_id')['count'].sum().reset_index(name='ville_categories_intersection')
ville_categories_total = ville_categories_total[ville_categories_total['ville_categories_intersection'] > 1]
# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(ville_categories_total, on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,ville_categories_intersection
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,856.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,467.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,95.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,516.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,160.0
...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,7.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,10.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,435.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,8.0


In [ ]:
# -----------------------------------------------------------
# 7) nb_restaurant_meme_annee
# Le nombre de restaurants qui sont ouverts leurs portes dans la même année que le restaurant en question.
# Ici, on considère que la première année d'un restaurant correspond à l'année de la première publication d'un avis sur ce restaurant sur Yelp.
# -----------------------------------------------------------

# Détermination de l'année de la première publication d'un avis pour chaque restaurant
annee_ouverture_restaurant = avis.groupby('restaurant_id')['annee'].min().reset_index(name='annee_ouverture')
# Comptage du nombre de restaurants ouverts chaque année
nb_restaurant_par_annee = annee_ouverture_restaurant['annee_ouverture'].value_counts().reset_index()
nb_restaurant_par_annee.columns = ['annee_ouverture', 'nb_restaurant_meme_annee']
# Fusion des données avec 'annee_ouverture_restaurant' pour associer chaque restaurant avec le nombre de restaurants ouverts la même année
restaurant_meme_annee = annee_ouverture_restaurant.merge(nb_restaurant_par_annee, on='annee_ouverture', how='left')
# Sélection des colonnes nécessaires
restaurant_meme_annee = restaurant_meme_annee[['restaurant_id', 'nb_restaurant_meme_annee']]
# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(restaurant_meme_annee, on='restaurant_id', how='left')
features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone','ville_categories_intersection','nb_restaurants_zone','nb_restaurant_meme_annee']].copy()
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,ville_categories_intersection,nb_restaurants_zone,nb_restaurant_meme_annee
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,856.0,49.0,3635
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,467.0,172.0,2392
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,95.0,77.0,3429
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,516.0,121.0,2993
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,160.0,96.0,2993
...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,7.0,53.0,3896
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,10.0,20.0,2802
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,435.0,72.0,3896
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,8.0,48.0,1852


In [ ]:
# -----------------------------------------------------------
# 8) ecart_type_etoiles
# L'écart type de la moyenne des étoiles par année.
# Il faut estimer la moyenne des étoiles par années. Puis, calculer l'écart type sur ces valeurs.
# -----------------------------------------------------------

# Extraction de l'année à partir de la date dans le DataFrame 'avis'
avis['annee'] = pd.to_datetime(avis['date']).dt.year
# Calcul de la moyenne des étoiles par année pour chaque restaurant
moyenne_etoiles_annee = avis.groupby(['restaurant_id', 'annee'])['etoiles'].mean().reset_index(name='moyenne_etoiles_annee')
# Calcul de l'écart type des moyennes d'étoiles pour chaque restaurant
ecart_type_etoiles = moyenne_etoiles_annee.groupby('restaurant_id')['moyenne_etoiles_annee'].std().reset_index(name='ecart_type_etoiles')
# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(ecart_type_etoiles, on='restaurant_id', how='left')
features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone','ville_categories_intersection','nb_restaurants_zone','nb_restaurant_meme_annee','ecart_type_etoiles']].copy()
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,ville_categories_intersection,nb_restaurants_zone,nb_restaurants_zone,nb_restaurant_meme_annee,ecart_type_etoiles
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,856.0,49.0,49.0,3635,0.771446
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,467.0,172.0,172.0,2392,0.066894
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,95.0,77.0,77.0,3429,0.578894
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,516.0,121.0,121.0,2993,0.593204
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,160.0,96.0,96.0,2993,0.139991
...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,7.0,53.0,53.0,3896,1.505545
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,10.0,20.0,20.0,2802,0.567238
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,435.0,72.0,72.0,3896,0.750925
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,8.0,48.0,48.0,1852,0.282843


In [ ]:



# -----------------------------------------------------------
# 9) tendance_etoiles
# La différence entre la moyenne des étoiles de la dernière année et la moyenne des étoiles de la première année d'un restaurant.
# Ici, on considère la première année d'un restaurant correspond à l'année de la première publication d'un avis sur ce restaurant sur Yelp.
# -----------------------------------------------------------

# Trouver la première année et la moyenne des étoiles correspondante pour chaque restaurant
premiere_annee = avis.groupby('restaurant_id').agg(premiere_annee=('annee', 'min'))
premiere_moyenne_etoiles = moyenne_etoiles_annee.merge(premiere_annee, left_on=['restaurant_id', 'annee'], right_on=['restaurant_id', 'premiere_annee'])
# Trouver la dernière année et la moyenne des étoiles correspondante pour chaque restaurant
derniere_annee = avis.groupby('restaurant_id').agg(derniere_annee=('annee', 'max'))
derniere_moyenne_etoiles = moyenne_etoiles_annee.merge(derniere_annee, left_on=['restaurant_id', 'annee'], right_on=['restaurant_id', 'derniere_annee'])
# Calcul de la tendance des étoiles (différence des moyennes)
tendance_etoiles = derniere_moyenne_etoiles.merge(premiere_moyenne_etoiles, on='restaurant_id', how='left')
tendance_etoiles['tendance_etoiles'] = tendance_etoiles['moyenne_etoiles_annee_x'] - tendance_etoiles['moyenne_etoiles_annee_y']
# Sélection des colonnes nécessaires
tendance_etoiles = tendance_etoiles[['restaurant_id', 'tendance_etoiles']]
# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(tendance_etoiles, on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,ville_categories_intersection,nb_restaurants_zone,nb_restaurants_zone,nb_restaurant_meme_annee,ecart_type_etoiles,tendance_etoiles
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,856.0,49.0,49.0,3635,0.771446,-0.826087
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,467.0,172.0,172.0,2392,0.066894,-0.005174
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,95.0,77.0,77.0,3429,0.578894,-1.666667
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,516.0,121.0,121.0,2993,0.593204,1.333333
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,160.0,96.0,96.0,2993,0.139991,-0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,7.0,53.0,53.0,3896,1.505545,2.000000
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,10.0,20.0,20.0,2802,0.567238,-1.083333
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,435.0,72.0,72.0,3896,0.750925,1.500000
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,8.0,48.0,48.0,1852,0.282843,-0.400000


In [ ]:
# -----------------------------------------------------------
# 10) nb_avis
# Le nombre total d'avis pour ce restaurant.
# -----------------------------------------------------------

# Groupement des données d'avis par 'restaurant_id' et comptage du nombre d'avis
nb_total_avis = avis.groupby('restaurant_id').size().reset_index(name='nb_total_avis')
# Fusion avec le DataFrame 'features' pour ajouter le nombre total d'avis
features = features.merge(nb_total_avis, on='restaurant_id', how='left')
features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone','ville_categories_intersection','nb_restaurants_zone','nb_restaurant_meme_annee','ecart_type_etoiles','tendance_etoiles','nb_total_avis']].copy()
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,ville_categories_intersection,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurant_meme_annee,ecart_type_etoiles,tendance_etoiles,nb_total_avis
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,856.0,49.0,49.0,49.0,49.0,3635,0.771446,-0.826087,72
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,467.0,172.0,172.0,172.0,172.0,2392,0.066894,-0.005174,108
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,95.0,77.0,77.0,77.0,77.0,3429,0.578894,-1.666667,49
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,516.0,121.0,121.0,121.0,121.0,2993,0.593204,1.333333,36
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,160.0,96.0,96.0,96.0,96.0,2993,0.139991,-0.200000,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,7.0,53.0,53.0,53.0,53.0,3896,1.505545,2.000000,6
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,10.0,20.0,20.0,20.0,20.0,2802,0.567238,-1.083333,11
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,435.0,72.0,72.0,72.0,72.0,3896,0.750925,1.500000,11
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,8.0,48.0,48.0,48.0,48.0,1852,0.282843,-0.400000,18


In [ ]:
# -----------------------------------------------------------
# 11) nb_avis_favorables
# Le nombre total d'avis favorables et positifs pour ce restaurant.
# On considère un avis "favorable" si son nombre d'étoiles est >=3.
# -----------------------------------------------------------

# Calcul du nombre total d'avis favorables pour chaque restaurant et fusion avec 'features'
nb_avis_favorables = avis[avis.etoiles >= 3].groupby('restaurant_id').size().reset_index(name='nb_avis_favorables')
features = features.merge(nb_avis_favorables, on='restaurant_id', how='left')
features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone','ville_categories_intersection','nb_restaurants_zone','nb_restaurant_meme_annee','ecart_type_etoiles','tendance_etoiles','nb_total_avis','nb_avis_favorables']].copy()
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurant_meme_annee,ecart_type_etoiles,tendance_etoiles,nb_total_avis,nb_avis_favorables
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,49.0,49.0,49.0,49.0,49.0,3635,0.771446,-0.826087,72,29.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,172.0,172.0,172.0,172.0,172.0,2392,0.066894,-0.005174,108,92.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,77.0,77.0,77.0,77.0,77.0,3429,0.578894,-1.666667,49,44.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,121.0,121.0,121.0,121.0,121.0,2993,0.593204,1.333333,36,29.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,96.0,96.0,96.0,96.0,96.0,2993,0.139991,-0.200000,88,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,53.0,53.0,53.0,53.0,53.0,3896,1.505545,2.000000,6,5.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,2802,0.567238,-1.083333,11,9.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,72.0,72.0,72.0,72.0,72.0,3896,0.750925,1.500000,11,10.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,48.0,48.0,48.0,48.0,48.0,1852,0.282843,-0.400000,18,16.0


In [ ]:
# -----------------------------------------------------------
# 12) nb_avis_defavorables
# Le nombre total d'avis défavorables pour ce restaurant.
# On considère un avis comme "défavorable" si son nombre d'étoiles est  < 3.
# -----------------------------------------------------------

# Calcul du nombre total d'avis defavorables pour chaque restaurant et fusion avec 'features'
nb_avis_defavorables = avis[avis.etoiles < 3].groupby('restaurant_id').size().reset_index(name='nb_avis_defavorables')
features = features.merge(nb_avis_defavorables, on='restaurant_id', how='left')
features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone','ville_categories_intersection','nb_restaurants_zone','nb_restaurant_meme_annee','ecart_type_etoiles','tendance_etoiles','nb_total_avis','nb_avis_favorables','nb_avis_defavorables']].copy()
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurant_meme_annee,ecart_type_etoiles,tendance_etoiles,nb_total_avis,nb_avis_favorables,nb_avis_defavorables
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,49.0,49.0,49.0,49.0,3635,0.771446,-0.826087,72,29.0,43.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,172.0,172.0,172.0,172.0,2392,0.066894,-0.005174,108,92.0,16.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,77.0,77.0,77.0,77.0,3429,0.578894,-1.666667,49,44.0,5.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,121.0,121.0,121.0,121.0,2993,0.593204,1.333333,36,29.0,7.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,96.0,96.0,96.0,96.0,2993,0.139991,-0.200000,88,77.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,53.0,53.0,53.0,53.0,3896,1.505545,2.000000,6,5.0,1.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,2802,0.567238,-1.083333,11,9.0,2.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,72.0,72.0,72.0,72.0,3896,0.750925,1.500000,11,10.0,1.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,48.0,48.0,48.0,48.0,1852,0.282843,-0.400000,18,16.0,2.0


In [ ]:
# -----------------------------------------------------------
# 13) ratio_avis_favorables
#  Le nombre d'avis favorables et positifs sur le nombre total d'avis pour ce restaurant.
# -----------------------------------------------------------

# Calcul du nombre total d'avis pour chaque restaurant
nb_total_avis = avis.groupby('restaurant_id').size().reset_index(name='nb_total_avis')
# Calcul du nombre d'avis favorables pour chaque restaurant (déjà fait auparavant)
# nb_avis_favorables = avis[avis.etoiles >= 3].groupby('restaurant_id').size().reset_index(name='nb_avis_favorables')
# Calcul du ratio des avis favorables sur le nombre total d'avis pour chaque restaurant
ratio_avis_favorables = nb_avis_favorables.merge(nb_total_avis, on='restaurant_id', how='left')
ratio_avis_favorables['ratio_avis_favorables'] = ratio_avis_favorables['nb_avis_favorables'] / ratio_avis_favorables['nb_total_avis']
# Fusion avec le DataFrame 'features' pour ajouter l'information du ratio
features = features.merge(ratio_avis_favorables[['restaurant_id', 'ratio_avis_favorables']], on='restaurant_id', how='left')
features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone','ville_categories_intersection','nb_restaurants_zone','nb_restaurant_meme_annee','ecart_type_etoiles','tendance_etoiles','nb_total_avis','nb_avis_favorables','nb_avis_defavorables','ratio_avis_favorables']].copy()
features


,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurant_meme_annee,ecart_type_etoiles,tendance_etoiles,nb_total_avis,nb_avis_favorables,nb_avis_defavorables,ratio_avis_favorables
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,49.0,49.0,49.0,3635,0.771446,-0.826087,72,29.0,43.0,0.402778
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,172.0,172.0,172.0,2392,0.066894,-0.005174,108,92.0,16.0,0.851852
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,77.0,77.0,77.0,3429,0.578894,-1.666667,49,44.0,5.0,0.897959
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,121.0,121.0,121.0,2993,0.593204,1.333333,36,29.0,7.0,0.805556
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,96.0,96.0,96.0,2993,0.139991,-0.200000,88,77.0,11.0,0.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,53.0,53.0,53.0,3896,1.505545,2.000000,6,5.0,1.0,0.833333
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,2802,0.567238,-1.083333,11,9.0,2.0,0.818182
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,72.0,72.0,72.0,3896,0.750925,1.500000,11,10.0,1.0,0.909091
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,48.0,48.0,48.0,1852,0.282843,-0.400000,18,16.0,2.0,0.888889


In [ ]:
# -----------------------------------------------------------
# 14) ratio_avis_defavorables
#  Le nombre d'avis défavorables sur le nombre total d'avis pour ce restaurant.
# -----------------------------------------------------------

# Calcul du nombre d'avis défavorables pour chaque restaurant
# nb_avis_defavorables = avis[avis.etoiles < 3].groupby('restaurant_id').size().reset_index(name='nb_avis_defavorables')
# Calcul du ratio des avis défavorables sur le nombre total d'avis pour chaque restaurant
ratio_avis_defavorables = nb_avis_defavorables.merge(nb_total_avis, on='restaurant_id', how='left')
ratio_avis_defavorables['ratio_avis_defavorables'] = ratio_avis_defavorables['nb_avis_defavorables'] / ratio_avis_defavorables['nb_total_avis']
# Fusion avec le DataFrame 'features' pour ajouter l'information du ratio
features = features.merge(ratio_avis_defavorables[['restaurant_id', 'ratio_avis_defavorables']], on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_restaurants_zone,nb_restaurants_zone,nb_restaurant_meme_annee,ecart_type_etoiles,tendance_etoiles,nb_total_avis,nb_avis_favorables,nb_avis_defavorables,ratio_avis_favorables,ratio_avis_defavorables
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,49.0,49.0,3635,0.771446,-0.826087,72,29.0,43.0,0.402778,0.597222
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,172.0,172.0,2392,0.066894,-0.005174,108,92.0,16.0,0.851852,0.148148
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,77.0,77.0,3429,0.578894,-1.666667,49,44.0,5.0,0.897959,0.102041
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,121.0,121.0,2993,0.593204,1.333333,36,29.0,7.0,0.805556,0.194444
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,96.0,96.0,2993,0.139991,-0.200000,88,77.0,11.0,0.875000,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,53.0,53.0,3896,1.505545,2.000000,6,5.0,1.0,0.833333,0.166667
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,2802,0.567238,-1.083333,11,9.0,2.0,0.818182,0.181818
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,72.0,72.0,3896,0.750925,1.500000,11,10.0,1.0,0.909091,0.090909
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,48.0,48.0,1852,0.282843,-0.400000,18,16.0,2.0,0.888889,0.111111


In [ ]:
# -----------------------------------------------------------
# 15) nb_avis_favorables_mention
# Le nombre total d'avis qui ont reçu au moins une mention "useful" ou "funny" ou "cool" ET le nombre d'étoiles de l'avis est >=3.
# -----------------------------------------------------------

# Calcul du nombre total d'avis favorables avec mention
# Un avis est considéré comme favorable s'il a un nombre d'étoiles >= 3
# et au moins une mention "useful", "funny" ou "cool"
condition_favorable = (avis.etoiles >= 3)
condition_mention = (avis.useful > 0) | (avis.funny > 0) | (avis.cool > 0)
nb_avis_favorables_mention = avis[condition_favorable & condition_mention].groupby('restaurant_id').size().reset_index(name='nb_avis_favorables_mention')
# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(nb_avis_favorables_mention, on='restaurant_id', how='left')
features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone','ville_categories_intersection','nb_restaurants_zone','nb_restaurant_meme_annee','ecart_type_etoiles','tendance_etoiles','nb_total_avis','nb_avis_favorables','nb_avis_defavorables','ratio_avis_favorables','ratio_avis_defavorables','nb_avis_favorables_mention']].copy()
features


,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_restaurants_zone,nb_restaurant_meme_annee,ecart_type_etoiles,tendance_etoiles,nb_total_avis,nb_avis_favorables,nb_avis_defavorables,ratio_avis_favorables,ratio_avis_defavorables,nb_avis_favorables_mention
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,49.0,3635,0.771446,-0.826087,72,29.0,43.0,0.402778,0.597222,8.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,172.0,2392,0.066894,-0.005174,108,92.0,16.0,0.851852,0.148148,42.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,77.0,3429,0.578894,-1.666667,49,44.0,5.0,0.897959,0.102041,24.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,121.0,2993,0.593204,1.333333,36,29.0,7.0,0.805556,0.194444,26.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,96.0,2993,0.139991,-0.200000,88,77.0,11.0,0.875000,0.125000,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,53.0,3896,1.505545,2.000000,6,5.0,1.0,0.833333,0.166667,1.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,20.0,2802,0.567238,-1.083333,11,9.0,2.0,0.818182,0.181818,4.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,72.0,3896,0.750925,1.500000,11,10.0,1.0,0.909091,0.090909,9.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,48.0,1852,0.282843,-0.400000,18,16.0,2.0,0.888889,0.111111,12.0


In [ ]:
# -----------------------------------------------------------
# 16) nb_avis_defavorables_mention
# Le nombre total d'avis qui ont reçu au moins une mention "useful" ou "funny" ou "cool" ET le nombre d'étoiles de l'avis est <3.
# -----------------------------------------------------------

# Calcul du nombre total d'avis défavorables avec mention
# Un avis est considéré comme défavorable s'il a un nombre d'étoiles < 3
# et au moins une mention "useful", "funny" ou "cool"
condition_defavorable = (avis.etoiles < 3)
condition_mention = (avis.useful > 0) | (avis.funny > 0) | (avis.cool > 0)
nb_avis_defavorables_mention = avis[condition_defavorable & condition_mention].groupby('restaurant_id').size().reset_index(name='nb_avis_defavorables_mention')
# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(nb_avis_defavorables_mention, on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_restaurant_meme_annee,ecart_type_etoiles,tendance_etoiles,nb_total_avis,nb_avis_favorables,nb_avis_defavorables,ratio_avis_favorables,ratio_avis_defavorables,nb_avis_favorables_mention,nb_avis_defavorables_mention
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,3635,0.771446,-0.826087,72,29.0,43.0,0.402778,0.597222,8.0,17.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,2392,0.066894,-0.005174,108,92.0,16.0,0.851852,0.148148,42.0,9.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,3429,0.578894,-1.666667,49,44.0,5.0,0.897959,0.102041,24.0,3.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,2993,0.593204,1.333333,36,29.0,7.0,0.805556,0.194444,26.0,4.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,2993,0.139991,-0.200000,88,77.0,11.0,0.875000,0.125000,35.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,3896,1.505545,2.000000,6,5.0,1.0,0.833333,0.166667,1.0,NaN
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,2802,0.567238,-1.083333,11,9.0,2.0,0.818182,0.181818,4.0,2.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,3896,0.750925,1.500000,11,10.0,1.0,0.909091,0.090909,9.0,NaN
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,1852,0.282843,-0.400000,18,16.0,2.0,0.888889,0.111111,12.0,1.0


In [ ]:
# -----------------------------------------------------------
# 17) nb_avis_favorables_elites
# Le nombre total d'avis favorables pour un restaurant qui sont rédigés par des utilisateurs élites.
# Dans ce travail, on considère un utilisateur élite si son statut est "élite" (élite = 1 dans la table Utilisateurs)
# ET il a rédigé au moins 100 avis au total ET il a au moins 100 avis avec mention.
# -----------------------------------------------------------

# Filtrage des avis favorables (étoiles >= 3)
condition_avis_favorable = (avis.etoiles >= 3)
# Filtrage des avis avec mention (useful, funny, ou cool > 0)
condition_avis_mention = (avis.useful > 0) | (avis.funny > 0) | (avis.cool > 0)
# Filtrage des utilisateurs élites
condition_utilisateur_elite = (utilisateurs.elite == 1) & (utilisateurs.nb_avis >= 100) & (utilisateurs.nb_avis_mention >= 100)
# Sélection des avis favorables avec mention rédigés par des utilisateurs élites
avis_elites = avis[condition_avis_favorable & condition_avis_mention].merge(utilisateurs[condition_utilisateur_elite], on='utilisateur_id', how='left')
# Calcul du nombre total d'avis favorables élites par restaurant
nb_avis_favorables_elites = avis_elites.groupby('restaurant_id').size().reset_index(name='nb_avis_favorables_elites')
# Fusion avec le DataFrame 'features'
features = features.merge(nb_avis_favorables_elites, on='restaurant_id', how='left')
features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone','ville_categories_intersection','nb_restaurants_zone','nb_restaurant_meme_annee','ecart_type_etoiles','tendance_etoiles','nb_total_avis','nb_avis_favorables','nb_avis_defavorables','ratio_avis_favorables','ratio_avis_defavorables','nb_avis_favorables_mention','nb_avis_defavorables_mention','nb_avis_favorables_elites']].copy()
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,ecart_type_etoiles,tendance_etoiles,nb_total_avis,nb_avis_favorables,nb_avis_defavorables,ratio_avis_favorables,ratio_avis_defavorables,nb_avis_favorables_mention,nb_avis_defavorables_mention,nb_avis_favorables_elites
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,0.771446,-0.826087,72,29.0,43.0,0.402778,0.597222,8.0,17.0,8.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,0.066894,-0.005174,108,92.0,16.0,0.851852,0.148148,42.0,9.0,42.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,0.578894,-1.666667,49,44.0,5.0,0.897959,0.102041,24.0,3.0,24.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,0.593204,1.333333,36,29.0,7.0,0.805556,0.194444,26.0,4.0,26.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,0.139991,-0.200000,88,77.0,11.0,0.875000,0.125000,35.0,6.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,1.505545,2.000000,6,5.0,1.0,0.833333,0.166667,1.0,NaN,1.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,0.567238,-1.083333,11,9.0,2.0,0.818182,0.181818,4.0,2.0,4.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,0.750925,1.500000,11,10.0,1.0,0.909091,0.090909,9.0,NaN,9.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,0.282843,-0.400000,18,16.0,2.0,0.888889,0.111111,12.0,1.0,12.0


In [ ]:
# -----------------------------------------------------------
# 18) nb_avis_defavorables_elites
# Le nombre total d'avis défavorables pour un restaurant qui sont rédigés par des utilisateurs élites.
# Dans ce travail, on considère un utilisateur élite si son statut est "élite" (élite = 1 dans la table Utilisateurs)
# ET il a rédigé au moins 100 avis au total ET il a au moins 100 avis avec mention.
# -----------------------------------------------------------

# Filtrage des avis défavorables (étoiles < 3)
condition_avis_defavorable = (avis.etoiles < 3)
# Filtrage des avis avec mention (useful, funny, ou cool > 0)
condition_avis_mention = (avis.useful > 1) | (avis.funny > 1) | (avis.cool > 1 )
# Filtrage des utilisateurs élites
condition_utilisateur_elite = (utilisateurs.elite == 1) & (utilisateurs.nb_avis >= 100) & (utilisateurs.nb_avis_mention >= 100)
# Sélection des avis défavorables avec mention rédigés par des utilisateurs élites
avis_elites_defavorables = avis[condition_avis_defavorable & condition_avis_mention!=1].merge(utilisateurs[condition_utilisateur_elite], on='utilisateur_id', how='left')
# Calcul du nombre total d'avis défavorables élites par restaurant
nb_avis_defavorables_elites = avis_elites_defavorables.groupby('restaurant_id').size().reset_index(name='nb_avis_defavorables_elites')
# Fusion avec le DataFrame 'features'
features = features.merge(nb_avis_defavorables_elites, on='restaurant_id', how='left')
#features = features[['restaurant_id', 'moyenne_etoiles', 'ville', 'zone', 'ferme','nb_restaurants_zone','ville_categories_intersection','nb_restaurants_zone','nb_restaurant_meme_annee','ecart_type_etoiles','tendance_etoiles','nb_total_avis','nb_avis_favorables','nb_avis_defavorables','ratio_avis_favorables','ratio_avis_defavorables','nb_avis_favorables_mention','nb_avis_defavorables_mention','nb_avis_favorables_elites','nb_avis_defavorables_elites']].copy()
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,tendance_etoiles,nb_total_avis,nb_avis_favorables,nb_avis_defavorables,ratio_avis_favorables,ratio_avis_defavorables,nb_avis_favorables_mention,nb_avis_defavorables_mention,nb_avis_favorables_elites,nb_avis_defavorables_elites
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,-0.826087,72,29.0,43.0,0.402778,0.597222,8.0,17.0,8.0,65.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,-0.005174,108,92.0,16.0,0.851852,0.148148,42.0,9.0,42.0,105.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,-1.666667,49,44.0,5.0,0.897959,0.102041,24.0,3.0,24.0,47.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,1.333333,36,29.0,7.0,0.805556,0.194444,26.0,4.0,26.0,34.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,-0.200000,88,77.0,11.0,0.875000,0.125000,35.0,6.0,35.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,2.000000,6,5.0,1.0,0.833333,0.166667,1.0,NaN,1.0,6.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,-1.083333,11,9.0,2.0,0.818182,0.181818,4.0,2.0,4.0,10.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,1.500000,11,10.0,1.0,0.909091,0.090909,9.0,NaN,9.0,11.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,-0.400000,18,16.0,2.0,0.888889,0.111111,12.0,1.0,12.0,17.0


# Nouvelle section

In [ ]:
# -----------------------------------------------------------
# 19) nb_conseils
#  Le nombre total de conseils (tips) associés à un restaurant.
# -----------------------------------------------------------

# Calcul du nombre total de conseils par restaurant
nb_conseils = conseils.groupby('restaurant_id').size().reset_index(name='nb_conseils')
# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(nb_conseils, on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_total_avis,nb_avis_favorables,nb_avis_defavorables,ratio_avis_favorables,ratio_avis_defavorables,nb_avis_favorables_mention,nb_avis_defavorables_mention,nb_avis_favorables_elites,nb_avis_defavorables_elites,nb_conseils
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,72,29.0,43.0,0.402778,0.597222,8.0,17.0,8.0,65.0,20.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,108,92.0,16.0,0.851852,0.148148,42.0,9.0,42.0,105.0,16.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,49,44.0,5.0,0.897959,0.102041,24.0,3.0,24.0,47.0,9.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,36,29.0,7.0,0.805556,0.194444,26.0,4.0,26.0,34.0,3.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,88,77.0,11.0,0.875000,0.125000,35.0,6.0,35.0,87.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,6,5.0,1.0,0.833333,0.166667,1.0,NaN,1.0,6.0,4.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,11,9.0,2.0,0.818182,0.181818,4.0,2.0,4.0,10.0,1.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,11,10.0,1.0,0.909091,0.090909,9.0,NaN,9.0,11.0,9.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,18,16.0,2.0,0.888889,0.111111,12.0,1.0,12.0,17.0,3.0


In [ ]:
# -----------------------------------------------------------
# 20) nb_conseils_compliment
# Le nombre total de conseils qui ont reçu au moins un compliment (voir Table Conseils).
# -----------------------------------------------------------

# Filtrage des conseils ayant reçu au moins un compliment
conseils_avec_compliment = conseils[conseils.nb_compliments > 0]
# Calcul du nombre total de conseils avec compliment par restaurant
nb_conseils_compliment = conseils_avec_compliment.groupby('restaurant_id').size().reset_index(name='nb_conseils_compliment')
# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(nb_conseils_compliment, on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_avis_favorables,nb_avis_defavorables,ratio_avis_favorables,ratio_avis_defavorables,nb_avis_favorables_mention,nb_avis_defavorables_mention,nb_avis_favorables_elites,nb_avis_defavorables_elites,nb_conseils,nb_conseils_compliment
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,29.0,43.0,0.402778,0.597222,8.0,17.0,8.0,65.0,20.0,NaN
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,92.0,16.0,0.851852,0.148148,42.0,9.0,42.0,105.0,16.0,1.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,44.0,5.0,0.897959,0.102041,24.0,3.0,24.0,47.0,9.0,NaN
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,29.0,7.0,0.805556,0.194444,26.0,4.0,26.0,34.0,3.0,NaN
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,77.0,11.0,0.875000,0.125000,35.0,6.0,35.0,87.0,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,5.0,1.0,0.833333,0.166667,1.0,NaN,1.0,6.0,4.0,NaN
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,9.0,2.0,0.818182,0.181818,4.0,2.0,4.0,10.0,1.0,NaN
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,10.0,1.0,0.909091,0.090909,9.0,NaN,9.0,11.0,9.0,NaN
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,16.0,2.0,0.888889,0.111111,12.0,1.0,12.0,17.0,3.0,NaN


In [ ]:
# -----------------------------------------------------------
# 21) nb_conseils_elites
# Le nombre total de conseils sur un restaurant qui sont rédigés par des utilisateurs élites.
# Dans ce travail, on considère un utilisateur élite si son statut est "élite" (élite = 1 dans la table Utilisateurs)
# ET il a rédigé au moins 100 avis au total ET il a au moins 100 avis avec mention.
# -----------------------------------------------------------

# Filtrage des utilisateurs élites
utilisateurs_elites = utilisateurs[(utilisateurs.elite == 1) & (utilisateurs.nb_avis >= 100) & (utilisateurs.nb_avis_mention >= 100)]
# Fusion de 'conseils' avec les utilisateurs élites
nb_conseils_elites = conseils.merge(utilisateurs_elites[['utilisateur_id']], on='utilisateur_id', how='inner') # icicicicicicicici (left insted of inner)
# Groupement par 'restaurant_id' et comptage des conseils
nb_conseils_elites = nb_conseils_elites.groupby('restaurant_id').size().reset_index(name='nb_conseils_elites')
# Fusion avec le DataFrame 'features'
features = features.merge(nb_conseils_elites, on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_avis_defavorables,ratio_avis_favorables,ratio_avis_defavorables,nb_avis_favorables_mention,nb_avis_defavorables_mention,nb_avis_favorables_elites,nb_avis_defavorables_elites,nb_conseils,nb_conseils_compliment,nb_conseils_elites
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,43.0,0.402778,0.597222,8.0,17.0,8.0,65.0,20.0,NaN,2.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,16.0,0.851852,0.148148,42.0,9.0,42.0,105.0,16.0,1.0,2.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,5.0,0.897959,0.102041,24.0,3.0,24.0,47.0,9.0,NaN,3.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,7.0,0.805556,0.194444,26.0,4.0,26.0,34.0,3.0,NaN,NaN
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,11.0,0.875000,0.125000,35.0,6.0,35.0,87.0,5.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,1.0,0.833333,0.166667,1.0,NaN,1.0,6.0,4.0,NaN,NaN
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,2.0,0.818182,0.181818,4.0,2.0,4.0,10.0,1.0,NaN,NaN
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,1.0,0.909091,0.090909,9.0,NaN,9.0,11.0,9.0,NaN,NaN
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,2.0,0.888889,0.111111,12.0,1.0,12.0,17.0,3.0,NaN,NaN


In [ ]:
# -----------------------------------------------------------
# 22) nb_checkin
# Le nombre total de visites.
# -----------------------------------------------------------

# Groupement des données de check-in par 'restaurant_id' et comptage du nombre de check-ins
nb_checkin = checkin.groupby('restaurant_id').size().reset_index(name='nb_checkin')
# Fusion avec le DataFrame 'features' pour ajouter le nombre total de check-ins
features = features.merge(nb_checkin, on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,ratio_avis_favorables,ratio_avis_defavorables,nb_avis_favorables_mention,nb_avis_defavorables_mention,nb_avis_favorables_elites,nb_avis_defavorables_elites,nb_conseils,nb_conseils_compliment,nb_conseils_elites,nb_checkin
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,0.402778,0.597222,8.0,17.0,8.0,65.0,20.0,NaN,2.0,181.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,0.851852,0.148148,42.0,9.0,42.0,105.0,16.0,1.0,2.0,492.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,0.897959,0.102041,24.0,3.0,24.0,47.0,9.0,NaN,3.0,157.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,0.805556,0.194444,26.0,4.0,26.0,34.0,3.0,NaN,NaN,18.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,0.875000,0.125000,35.0,6.0,35.0,87.0,5.0,NaN,2.0,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,0.833333,0.166667,1.0,NaN,1.0,6.0,4.0,NaN,NaN,37.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,0.818182,0.181818,4.0,2.0,4.0,10.0,1.0,NaN,NaN,5.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,0.909091,0.090909,9.0,NaN,9.0,11.0,9.0,NaN,NaN,32.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,0.888889,0.111111,12.0,1.0,12.0,17.0,3.0,NaN,NaN,7.0


In [ ]:
# -----------------------------------------------------------
# 23) moyenne_checkin
# La moyenne de visites par année.
# -----------------------------------------------------------

# Calculer le nombre total de visites par restaurant et par année
total_visites = checkin.groupby(['restaurant_id', 'annee']).size().reset_index(name='total_visites')
# Calculer la moyenne des visites par année pour chaque restaurant
moyenne_checkin = total_visites.groupby('restaurant_id').total_visites.mean().reset_index(name='moyenne_checkin')
# Fusionner cette moyenne avec le DataFrame 'features'
features = features.merge(moyenne_checkin[['restaurant_id', 'moyenne_checkin']], on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,ratio_avis_defavorables,nb_avis_favorables_mention,nb_avis_defavorables_mention,nb_avis_favorables_elites,nb_avis_defavorables_elites,nb_conseils,nb_conseils_compliment,nb_conseils_elites,nb_checkin,moyenne_checkin
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,0.597222,8.0,17.0,8.0,65.0,20.0,NaN,2.0,181.0,20.111111
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,0.148148,42.0,9.0,42.0,105.0,16.0,1.0,2.0,492.0,164.000000
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,0.102041,24.0,3.0,24.0,47.0,9.0,NaN,3.0,157.0,17.444444
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,0.194444,26.0,4.0,26.0,34.0,3.0,NaN,NaN,18.0,3.600000
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,0.125000,35.0,6.0,35.0,87.0,5.0,NaN,2.0,110.0,18.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,0.166667,1.0,NaN,1.0,6.0,4.0,NaN,NaN,37.0,4.625000
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,0.181818,4.0,2.0,4.0,10.0,1.0,NaN,NaN,5.0,2.500000
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,0.090909,9.0,NaN,9.0,11.0,9.0,NaN,NaN,32.0,5.333333
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,0.111111,12.0,1.0,12.0,17.0,3.0,NaN,NaN,7.0,3.500000


In [ ]:
# -----------------------------------------------------------
# 24) ecart_type_checkin
# L'écart type de visites par année.
# Ici, on calcul l'écart type pour le total des visites par année.
# -----------------------------------------------------------

# Calculer le nombre total de visites par restaurant et par année
total_visites = checkin.groupby(['restaurant_id', 'annee']).size().reset_index(name='total_visites')
# Calculer l'écart type des visites pour chaque restaurant
ecart_type_checkin = total_visites.groupby('restaurant_id').total_visites.std().reset_index(name='ecart_type_checkin')
# Fusionner cet écart type avec le DataFrame 'features'
features = features.merge(ecart_type_checkin[['restaurant_id', 'ecart_type_checkin']], on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_avis_favorables_mention,nb_avis_defavorables_mention,nb_avis_favorables_elites,nb_avis_defavorables_elites,nb_conseils,nb_conseils_compliment,nb_conseils_elites,nb_checkin,moyenne_checkin,ecart_type_checkin
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,8.0,17.0,8.0,65.0,20.0,NaN,2.0,181.0,20.111111,10.635371
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,42.0,9.0,42.0,105.0,16.0,1.0,2.0,492.0,164.000000,52.086467
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,24.0,3.0,24.0,47.0,9.0,NaN,3.0,157.0,17.444444,15.828069
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,26.0,4.0,26.0,34.0,3.0,NaN,NaN,18.0,3.600000,1.816590
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,35.0,6.0,35.0,87.0,5.0,NaN,2.0,110.0,18.333333,8.891944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,1.0,NaN,1.0,6.0,4.0,NaN,NaN,37.0,4.625000,5.553313
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,4.0,2.0,4.0,10.0,1.0,NaN,NaN,5.0,2.500000,2.121320
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,9.0,NaN,9.0,11.0,9.0,NaN,NaN,32.0,5.333333,6.282250
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,12.0,1.0,12.0,17.0,3.0,NaN,NaN,7.0,3.500000,0.707107


In [ ]:
# -----------------------------------------------------------
# 25) chaine
#  Prend 0 ou 1. La valeur 1 indique que le restaurant fait parti d'une chaîne (p. ex. McDonald).
#  On considère un restaurant comme il fait partie d'une chaîne, s’il existe un autre restaurant dans la base de données qui a le même nom.
# -----------------------------------------------------------

# Grouper les restaurants par nom et compter le nombre d'occurrences de chaque nom
noms_count = restaurants[['restaurant_id', 'nom']].groupby('nom').size().reset_index(name='nom_count')
# Définir la valeur de 'chaine' à 1 si un restaurant appartient à une chaîne (nombre d'occurrences > 1) et à 0 sinon
noms_count.loc[noms_count['nom_count'] > 1, 'chaine' ] = 1
noms_count.loc[noms_count['nom_count'] <= 1, 'chaine' ] = 0
# Fusionner cette information avec les données des restaurants
restaurants_chaine = restaurants[['restaurant_id', 'nom']].merge(noms_count[['nom', 'chaine']], on='nom', how='left')
# Fusionner cette information avec le DataFrame 'features'
features = features.merge(restaurants_chaine[['restaurant_id', 'chaine']], on='restaurant_id', how='left')
features

,restaurant_id,moyenne_etoiles,ville,zone,ferme,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,nb_restaurants_zone,...,nb_avis_defavorables_mention,nb_avis_favorables_elites,nb_avis_defavorables_elites,nb_conseils,nb_conseils_compliment,nb_conseils_elites,nb_checkin,moyenne_checkin,ecart_type_checkin,chaine
0,lCwqJWMxvIUQt1Re_tDn4w,2.5,Las Vegas,89110,0,49.0,49.0,49.0,49.0,49.0,...,17.0,8.0,65.0,20.0,NaN,2.0,181.0,20.111111,10.635371,1.0
1,pd0v6sOqpLhFJ7mkpIaixw,4.0,Phoenix,85004,0,172.0,172.0,172.0,172.0,172.0,...,9.0,42.0,105.0,16.0,1.0,2.0,492.0,164.000000,52.086467,1.0
2,0vhi__HtC2L4-vScgDFdFw,3.5,Calgary,T2T,0,77.0,77.0,77.0,77.0,77.0,...,3.0,24.0,47.0,9.0,NaN,3.0,157.0,17.444444,15.828069,0.0
3,t65yfB9v9fqlhAkLnnUXdg,3.5,Toronto,M5A,0,121.0,121.0,121.0,121.0,121.0,...,4.0,26.0,34.0,3.0,NaN,NaN,18.0,3.600000,1.816590,0.0
4,i7_JPit-2kAbtRTLkic2jA,4.0,Toronto,M5H,0,96.0,96.0,96.0,96.0,96.0,...,6.0,35.0,87.0,5.0,NaN,2.0,110.0,18.333333,8.891944,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,3.5,McMurray,15317,1,53.0,53.0,53.0,53.0,53.0,...,NaN,1.0,6.0,4.0,NaN,NaN,37.0,4.625000,5.553313,1.0
34264,Hq2edcOTjse7wjK2CwBijQ,3.5,Pointe-Claire,H9S,1,20.0,20.0,20.0,20.0,20.0,...,2.0,4.0,10.0,1.0,NaN,NaN,5.0,2.500000,2.121320,0.0
34265,7KlpgRjjAmVabPzxcExs0g,4.0,Phoenix,85051,1,72.0,72.0,72.0,72.0,72.0,...,NaN,9.0,11.0,9.0,NaN,NaN,32.0,5.333333,6.282250,0.0
34266,0fY-zYyP2fDmp2YXFsuNTg,4.0,Sun Prairie,53590,1,48.0,48.0,48.0,48.0,48.0,...,1.0,12.0,17.0,3.0,NaN,NaN,7.0,3.500000,0.707107,0.0


In [ ]:
# -----------------------------------------------------------
# 26) nb_heures_ouverture_semaine
# Le nombre total d'heures d'ouverture du restaurant par semaine.
# -----------------------------------------------------------

# Fonction pour convertir les plages horaires en nombre d'heures
def convertir_heures(plage_horaire):
    if pd.isna(plage_horaire):
      return 0

    if plage_horaire == "0:0-0:0":
      return 24

    debut, fin = plage_horaire.split('-')
    heure_debut, minute_debut = map(int, debut.split(':'))
    heure_fin, minute_fin = map(int, fin.split(':'))

    # Gérer le cas où l'heure de fermeture est après minuit (heure de début > heure de fin)
    if heure_fin < heure_debut or (heure_fin == heure_debut and minute_fin < minute_debut):
        heure_fin += 24

    duree = (heure_fin + minute_fin / 60) - (heure_debut + minute_debut / 60)

    # Gestion spéciale pour "1:0-1:0" et similaire, traité comme 12 heures d'ouverture par conséquent on ajoute +12 pour prendre le format 24h
    if debut == fin:
      duree += 12

    return duree

# Appliquer la conversion pour chaque jour et calculer le total hebdomadaire
jours = ['lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi', 'samedi', 'dimanche']
for jour in jours:
    horaires[jour] = horaires[jour].apply(convertir_heures)

horaires['nb_heures_ouverture_semaine'] = horaires[jours].sum(axis=1)

# Fusion avec le DataFrame 'features' pour ajouter cette information
features = features.merge(horaires[['restaurant_id', 'nb_heures_ouverture_semaine']], on='restaurant_id', how='left')



In [ ]:
# -----------------------------------------------------------
# 27) ouvert_samedi
# Si le restaurant est ouvert les samedis (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

horaires['ouvert_samedi'] = horaires['samedi'].apply(lambda x: 0 if pd.isna(x) or x == 0 else 1)
features = features.merge(horaires[['restaurant_id', 'ouvert_samedi']], on='restaurant_id', how='left')
features

# 27) ouvert_samedi
# Si le restaurant est ouvert les samedis (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------
horaires['ouvert_samedi'] = horaires['samedi_diff'].apply(lambda x: 1 if x >= 1 else 0 )
features = pd.merge(features,horaires[['restaurant_id','ouvert_samedi']], how='left',on='restaurant_id')


KeyError: 'samedi_diff'

In [ ]:
# -----------------------------------------------------------
# 28) ouvert_dimanche
# Si le restaurant est ouvert les dimanches (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

horaires['ouvert_dimanche'] = horaires['dimanche'].apply(lambda x: 0 if pd.isna(x) or x == 0 else 1)
features = features.merge(horaires[['restaurant_id', 'ouvert_dimanche']], on='restaurant_id', how='left')

In [ ]:

# -----------------------------------------------------------
# 29) ouvert_lundi
# Si le restaurant est ouvert les lundis (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

horaires['ouvert_lundi'] = horaires['lundi'].apply(lambda x: 0 if pd.isna(x) or x == 0 else 1)
features = features.merge(horaires[['restaurant_id', 'ouvert_lundi']], on='restaurant_id', how='left')

In [ ]:
# -----------------------------------------------------------
# 30) ouvert_vendredi
# Si le restaurant est ouvert les vendredis (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

horaires['ouvert_vendredi'] = horaires['vendredi'].apply(lambda x: 0 if pd.isna(x) or x == 0 else 1)
features = features.merge(horaires[['restaurant_id', 'ouvert_vendredi']], on='restaurant_id', how='left')

In [ ]:
# -----------------------------------------------------------
# 31) emporter
# Si le restaurant offre le service à emporter (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

features = features.merge(services[['restaurant_id', 'emporter']], on='restaurant_id', how='left')
features

In [ ]:
# -----------------------------------------------------------
# 32) livraison
# Si le restaurant offre le service de livraison (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

features = features.merge(services[['restaurant_id', 'livraison']], on='restaurant_id', how='left')

In [ ]:
# -----------------------------------------------------------
# 33) bon_pour_groupes
# Si le restaurant est approprié pour les groupes (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

features = features.merge(services[['restaurant_id', 'bon_pour_groupes']], on='restaurant_id', how='left')

In [ ]:
# -----------------------------------------------------------
# 34) bon_pour_enfants
# Si le restaurant est approprié pour les enfants (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

features = features.merge(services[['restaurant_id', 'bon_pour_enfants']], on='restaurant_id', how='left')

In [ ]:
# -----------------------------------------------------------
# 35) reservation
# Si on a besoin de faire une réservation au restaurant (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

features = features.merge(services[['restaurant_id', 'reservation']], on='restaurant_id', how='left')

In [ ]:
# -----------------------------------------------------------
# 36) prix
# Le niveau de prix du restaurant. Il existe trois niveaux, 1 (abordable), 2 (moyen) et 3 (coûteux).
# -----------------------------------------------------------

# Dictionnaire pour convertir les niveaux de prix
niveaux_prix = {1: 'abordable', 2: 'moyen', 3: 'coûteux'}
# Appliquer le dictionnaire pour transformer la colonne 'prix'
services['prix'] = services['prix'].map(niveaux_prix)
# Fusionner 'prix' avec 'features'
features = features.merge(services[['restaurant_id', 'prix']], on='restaurant_id', how='left')

In [ ]:

# -----------------------------------------------------------
# 37) terrasse
# Si le restaurant a une terrasse (valeur booléenne : 0 ou 1).
# -----------------------------------------------------------

features = features.merge(services[['restaurant_id', 'terrasse']], on='restaurant_id', how='left')

# -----------------------------------------------------------
# Sauvegarder l'ensemble de données dans un fichier csv afin d'être utilisé dans l'étape suivante
features.to_csv("donnees/features.csv", index=False)